##### Copyright 2018 Les auteurs de TensorFlow.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Points de contrôle de la formation

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/checkpoint"><img src="https://www.tensorflow.org/images/tf_logo_32px.png"> Voir sur TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png"> Exécuter dans Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"> Afficher la source sur GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png"> Télécharger le cahier</a></td>
</table>

L'expression "Enregistrement d'un modèle TensorFlow" signifie généralement l'une des deux choses suivantes:

1. Points de contrôle, OU
2. SavedModel.

Les points de contrôle capturent la valeur exacte de tous les paramètres (objets `tf.Variable` ) utilisés par un modèle. Les points de contrôle ne contiennent aucune description du calcul défini par le modèle et ne sont donc généralement utiles que lorsque le code source qui utilisera les valeurs de paramètres enregistrées est disponible.

Le format SavedModel, quant à lui, comprend une description sérialisée du calcul défini par le modèle en plus des valeurs des paramètres (point de contrôle). Les modèles dans ce format sont indépendants du code source qui a créé le modèle. Ils sont ainsi adaptés au déploiement via TensorFlow Serving, TensorFlow Lite, TensorFlow.js, ou des programmes dans d'autres langages de programmation (les API TensorFlow C, C ++, Java, Go, Rust, C # etc.).

Ce guide couvre les API pour l'écriture et la lecture des points de contrôle.

## Installer

In [0]:
import tensorflow as tf

In [0]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [0]:
net = Net()

## Enregistrement à partir des API de formation `tf.keras`

Consultez le [guide `tf.keras` sur l'enregistrement et la restauration](./keras/overview.ipynb#save_and_restore) .

`tf.keras.Model.save_weights` enregistre un point de contrôle TensorFlow. 

In [0]:
net.save_weights('easy_checkpoint')

## Écriture des points de contrôle


L'état persistant d'un modèle TensorFlow est stocké dans les objets `tf.Variable` . Ceux-ci peuvent être construits directement, mais sont souvent créés via des API de haut niveau telles que `tf.keras.layers` ou `tf.keras.Model` .

Le moyen le plus simple de gérer les variables consiste à les attacher à des objets Python, puis à référencer ces objets.

Les sous-classes de `tf.train.Checkpoint` , `tf.keras.layers.Layer` et `tf.keras.Model` automatiquement les variables affectées à leurs attributs. L'exemple suivant construit un modèle linéaire simple, puis écrit des points de contrôle qui contiennent des valeurs pour toutes les variables du modèle.

Vous pouvez facilement enregistrer un modèle-point de contrôle avec `Model.save_weights`

### Point de contrôle manuel

#### Installer

Pour aider à démontrer toutes les fonctionnalités de `tf.train.Checkpoint` définissez un jeu de données de jouet et une étape d'optimisation:

In [0]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)

In [0]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

#### Créer les objets de point de contrôle

Pour créer manuellement un point de contrôle, vous aurez besoin d'un objet `tf.train.Checkpoint` . Où les objets que vous souhaitez contrôler sont définis en tant qu'attributs sur l'objet.

Un `tf.train.CheckpointManager` peut également être utile pour gérer plusieurs points de contrôle.

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

#### Former et contrôler le modèle

La boucle d'apprentissage suivante crée une instance du modèle et d'un optimiseur, puis les rassemble dans un objet `tf.train.Checkpoint` . Il appelle l'étape d'entraînement en boucle sur chaque lot de données et écrit périodiquement des points de contrôle sur le disque.

In [0]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      print("loss {:1.2f}".format(loss.numpy()))

In [0]:
train_and_checkpoint(net, manager)

#### Restaurer et continuer la formation

Après le premier, vous pouvez passer un nouveau modèle et un nouveau gestionnaire, mais reprendre la formation exactement là où vous vous êtes arrêté:

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

L'objet `tf.train.CheckpointManager` supprime les anciens points de contrôle. Au-dessus, il est configuré pour ne conserver que les trois points de contrôle les plus récents.

In [0]:
print(manager.checkpoints)  # List the three remaining checkpoints

Ces chemins, par exemple `'./tf_ckpts/ckpt-10'` , ne sont pas des fichiers sur le disque. Au lieu de cela, ce sont des préfixes pour un fichier d' `index` et un ou plusieurs fichiers de données contenant les valeurs de variable. Ces préfixes sont regroupés dans un seul fichier de `checkpoint` ( `'./tf_ckpts/checkpoint'` ) dans lequel `CheckpointManager` enregistre son état.

In [0]:
!ls ./tf_ckpts

<a id="loading_mechanics"></a>

## Mécanique de chargement

TensorFlow fait correspondre les variables aux valeurs de points de contrôle en parcourant un graphe dirigé avec des arêtes nommées, en commençant par l'objet en cours de chargement. Les noms d'arêtes proviennent généralement des noms d'attributs dans les objets, par exemple le `"l1"` dans `self.l1 = tf.keras.layers.Dense(5)` . `tf.train.Checkpoint` utilise ses noms d'argument de mot-clé, comme dans `"step"` dans `tf.train.Checkpoint(step=...)` .

Le graphe de dépendances de l'exemple ci-dessus ressemble à ceci:

![Visualisation du graphe de dépendances pour l'exemple de boucle d'apprentissage](https://tensorflow.org/images/guide/whole_checkpoint.svg)

Avec l'optimiseur en rouge, les variables régulières en bleu et les variables d'emplacement de l'optimiseur en orange. Les autres nœuds, par exemple représentant le `tf.train.Checkpoint` , sont noirs.

Les variables d'emplacement font partie de l'état de l'optimiseur, mais sont créées pour une variable spécifique. Par exemple, les arêtes `'m'` ci-dessus correspondent à l'élan, que l'optimiseur Adam suit pour chaque variable. Les variables d'emplacement ne sont enregistrées dans un point de contrôle que si la variable et l'optimiseur sont tous deux enregistrés, donc les bords en pointillés.

L'appel de `restore()` sur un objet `tf.train.Checkpoint` file d'attente les restaurations demandées, restaurant les valeurs des variables dès qu'il existe un chemin correspondant à partir de l'objet `Checkpoint` . Par exemple, nous pouvons charger uniquement le biais du modèle que nous avons défini ci-dessus en reconstruisant un chemin vers celui-ci à travers le réseau et la couche.

In [0]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy())  # All zeros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('./tf_ckpts/'))
print(to_restore.numpy())  # We get the restored value now

Le graphe de dépendance pour ces nouveaux objets est un sous-graphe beaucoup plus petit du plus grand point de contrôle que nous avons écrit ci-dessus. Il comprend uniquement le biais et un compteur de sauvegarde que `tf.train.Checkpoint` utilise pour numéroter les points de contrôle.

![Visualisation d'un sous-graphe pour la variable de biais](https://tensorflow.org/images/guide/partial_checkpoint.svg)

`restore()` renvoie un objet status, qui a des assertions facultatives. Tous les objets que nous avons créés dans notre nouveau `Checkpoint` ont été restaurés, donc `status.assert_existing_objects_matched()` passe.

In [0]:
status.assert_existing_objects_matched()

Il y a de nombreux objets dans le point de contrôle qui ne correspondent pas, y compris le noyau de la couche et les variables de l'optimiseur. `status.assert_consumed()` ne passe que si le point de contrôle et le programme correspondent exactement, et lèverait une exception ici.

### Restaurations retardées

`Layer` objects in TensorFlow may delay the creation of variables to their first call, when input shapes are available. For example the shape of a `Dense` layer's kernel depends on both the layer's input and output shapes, and so the output shape required as a constructor argument is not enough information to create the variable on its own. Since calling a `Layer` also reads the variable's value, a restore must happen between the variable's creation and its first use.

Pour prendre en charge cet idiome, `tf.train.Checkpoint` restaure les files d'attente qui n'ont pas encore de variable correspondante.

In [0]:
delayed_restore = tf.Variable(tf.zeros([1, 5]))
print(delayed_restore.numpy())  # Not restored; still zeros
fake_layer.kernel = delayed_restore
print(delayed_restore.numpy())  # Restored

### Inspection manuelle des points de contrôle

`tf.train.list_variables` répertorie les clés de point de contrôle et les formes des variables dans un point de contrôle. Les clés de point de contrôle sont des chemins dans le graphique affiché ci-dessus.

In [0]:
tf.train.list_variables(tf.train.latest_checkpoint('./tf_ckpts/'))

### Suivi des listes et des dictionnaires

Comme pour les attributions directes d'attributs comme `self.l1 = tf.keras.layers.Dense(5)` , l'affectation de listes et de dictionnaires à des attributs suivra leur contenu.

In [0]:
save = tf.train.Checkpoint()
save.listed = [tf.Variable(1.)]
save.listed.append(tf.Variable(2.))
save.mapped = {'one': save.listed[0]}
save.mapped['two'] = save.listed[1]
save_path = save.save('./tf_list_example')

restore = tf.train.Checkpoint()
v2 = tf.Variable(0.)
assert 0. == v2.numpy()  # Not restored yet
restore.mapped = {'two': v2}
restore.restore(save_path)
assert 2. == v2.numpy()

Vous remarquerez peut-être des objets wrapper pour les listes et les dictionnaires. Ces wrappers sont des versions vérifiables des structures de données sous-jacentes. Tout comme le chargement basé sur les attributs, ces wrappers restaurent la valeur d'une variable dès qu'elle est ajoutée au conteneur.

In [0]:
restore.listed = []
print(restore.listed)  # ListWrapper([])
v1 = tf.Variable(0.)
restore.listed.append(v1)  # Restores v1, from restore() in the previous cell
assert 1. == v1.numpy()

Le même suivi est automatiquement appliqué aux sous-classes de `tf.keras.Model` , et peut être utilisé par exemple pour suivre des listes de couches.

## Enregistrement de points de contrôle basés sur des objets avec Estimator

Consultez le [guide Estimator](https://www.tensorflow.org/guide/estimator) .

Les estimateurs enregistrent par défaut les points de contrôle avec des noms de variables plutôt que le graphe d'objets décrit dans les sections précédentes. `tf.train.Checkpoint` acceptera les points de contrôle basés sur les noms, mais les noms de variables peuvent changer lors du déplacement de parties d'un modèle en dehors de `model_fn` de l'estimateur. L'enregistrement de points de contrôle basés sur des objets facilite l'apprentissage d'un modèle dans un Estimator, puis son utilisation en dehors de celui-ci.

In [0]:
import tensorflow.compat.v1 as tf_compat

In [0]:
def model_fn(features, labels, mode):
  net = Net()
  opt = tf.keras.optimizers.Adam(0.1)
  ckpt = tf.train.Checkpoint(step=tf_compat.train.get_global_step(),
                             optimizer=opt, net=net)
  with tf.GradientTape() as tape:
    output = net(features['x'])
    loss = tf.reduce_mean(tf.abs(output - features['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  return tf.estimator.EstimatorSpec(
    mode,
    loss=loss,
    train_op=tf.group(opt.apply_gradients(zip(gradients, variables)),
                      ckpt.step.assign_add(1)),
    # Tell the Estimator to save "ckpt" in an object-based format.
    scaffold=tf_compat.train.Scaffold(saver=ckpt))

tf.keras.backend.clear_session()
est = tf.estimator.Estimator(model_fn, './tf_estimator_example/')
est.train(toy_dataset, steps=10)

`tf.train.Checkpoint` peut alors charger les points de contrôle de l'estimateur à partir de son `model_dir` .

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(
  step=tf.Variable(1, dtype=tf.int64), optimizer=opt, net=net)
ckpt.restore(tf.train.latest_checkpoint('./tf_estimator_example/'))
ckpt.step.numpy()  # From est.train(..., steps=10)

## Résumé

Les objets TensorFlow fournissent un mécanisme automatique simple pour enregistrer et restaurer les valeurs des variables qu'ils utilisent.
